In [2]:
import pandas as pd

In [53]:
df = pd.read_parquet("fhv_tripdata_2021-01.parquet")

In [54]:
df.shape

(1154112, 7)

In [55]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


In [56]:
df = df.fillna(-1)

In [57]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [58]:
df.duration.mean()

19.1672240937939

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [59]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,-1,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,-1.0,-1.0,-1,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,-1,B00037,15.216667
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,-1,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,-1,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,-1,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,-1,B03285,19.433333


In [10]:
len(df[["PUlocationID","DOlocationID"]][(df.PUlocationID==-1)])/len(df)

0.8303067639882438

In [36]:
from sklearn.linear_model import LinearRegression


In [100]:
from sklearn.preprocessing import OneHotEncoder

In [101]:
ohe = OneHotEncoder()

In [ ]:
df["categ"] = df['PUlocationID'].astype(str) + '_' + df['DOlocationID'].astype(str)

In [102]:
X_train = ohe.fit_transform(df[["DOlocationID","PUlocationID"]].astype(str))

In [103]:
X_train.shape

(1154112, 525)

In [80]:
lr = LinearRegression()

In [81]:
lr.fit(X_train, df.duration.values)

LinearRegression()

In [82]:
from sklearn.metrics import mean_squared_error

In [83]:
mean_squared_error(df.duration, lr.predict(X_train), squared=False)

398.41649150202346

In [76]:
lr.predict(X_train)

array([33.32149116, 33.32149116, 33.32149116, ..., 16.03637313,
        8.95521588, 33.32149116])

In [68]:
df.duration

0           17.000000
1           17.000000
2          110.000000
3            8.283333
4           15.216667
              ...    
1154107      8.750000
1154108     57.600000
1154109     16.200000
1154110     19.433333
1154111     36.000000
Name: duration, Length: 1154112, dtype: float64